# Argumenty funkcí, Generátory a další

Tento notebook pokrývá obsahem část šesté přednášky z Programování v Jazyce Python v ZS 2017 na FM TUL. Přečíst si ho ale může každý, kdo se chce dozvědět něco víc o argumentech funkcí, tajemných slovech \*args a \*\*kwargs, vedlejších efektech in-place operací v Pythonu a v neposlední řadě také o generátorech. 


## Argumenty funkce

Argumenty funkce jsou v těle funkce přístupné nejen pod definovanými jmény, ale i všechny najednou. Slouží k tomu funkce locals(). Ta nám vrátí slovník, obsahující všechny argumenty funkce. Klíče ve slovníku jsou jména, která jsme argumentům přidělili při definování funkce.



In [277]:
def simple(a, b, c):
    print(locals())

In [278]:
simple(10, 10, 30)

{'c': 30, 'b': 10, 'a': 10}


Argumenty funkcí mají jména. Pokud jména použijeme při volání funkce, nemusíme nutně dodržet pořadí argumentů. Python si poradí s následujícím zápisem.

In [279]:
simple(c=4, a=4, b="222asdada")

{'c': 4, 'b': '222asdada', 'a': 4}


Argumenty mohou mít nějakou **výchozí hodnotu**. Ta se automaticky použije v případě, že argument při volání vynecháme. 

In [280]:
def implicit(a, b=10, c='retezec'):
    print(locals())

Takovou funkci potom můžeme zavolat pouze s jedním parametrem. Parametr **a** nemá výchozí hodnotu a proto musí být uveden, jinak volání funkce vyvolá TypeError.

In [281]:
implicit(2222)

{'c': 'retezec', 'b': 10, 'a': 2222}


Kombinací dvou předchozích příkladů dojdeme k tomu, že nemusíme uvádět všechny argumenty. Argument **b** vynecháme následujícím způsobem.

In [282]:
implicit(2222, c=222311)

{'c': 222311, 'b': 10, 'a': 2222}


## Hvězdičková magie

Následující zápis jste už mohli vidět v některých předchozích ukázkách. Nyní si vysvětlíme co přesně dělá. Je to možná trochu magické, ale ne složité.

Znak ** \* ** před argumentem **args** znamená, že funkce může přijmout libovolný počet nepojmenovaných argumentů. Obdobně pak dvojice ** \*\* ** před slovem **kwargs** umožňuje naší funkci přijmout libovolný počet pojmenovaných argumentů.

In [283]:
def magic(*args, **kwargs):
    print(locals())

Následující příklad ukazuje, že všechna čísla (nepojmenované argumenty) jsou uvnitř funkce přístupné právě pod klíčem args. Z toho je zřejmé, že jako mnoho jiných věcí jsou v Pythonu i jména **args** a **kwargs** pouze konvencí. Kterou ale opět doporučuji dodržovat.

In [284]:
magic(10, 20, 30, 40)

{'kwargs': {}, 'args': (10, 20, 30, 40)}


Totéž platí i pro argumenty se jmény.

In [285]:
magic(40, a=10, b=20, test=30)

{'kwargs': {'a': 10, 'b': 20, 'test': 30}, 'args': (40,)}


Hvězdičková magie funguje i v obráceném směru. Sekvenční typ jako je list či tupple můžete rozbalit při volání funkce.

Nejprve předáme funkci tupple jako celek. Celkem logicky je přístupný jako první prvek pod klíčem args.

tu = (10, 20, 30, 40)

In [286]:
magic(tu)

{'kwargs': {}, 'args': ((10, 20, 30, 40),)}


Pokud ale použijeme hvězdičku před jménem proměnné při volání funkce dojde k rozbalení obsahu sekvence na jednotlivé položky. Ty jsou pak přístupné buď jako args.

In [287]:
magic(*tu)

{'kwargs': {}, 'args': (10, 20, 30, 40)}


Nebo jako kwargs pokud použijeme k rozbalení slovník.

In [288]:
sl = {'2': 'jedno', 'keyword': 'zopakovani'}

In [289]:
magic(**sl)

{'kwargs': {'2': 'jedno', 'keyword': 'zopakovani'}, 'args': ()}


Funguje to i pro "normální" funkce - tedy funkce které přijímají předem daný počet parametrů. Jako je například funkce **simple**, kterou již máme vytvořenou.

In [290]:
tu2 = (10, 20, 30)
simple(*tu2)

{'c': 30, 'b': 20, 'a': 10}


V tomto případě ale musí mít sekvence správný počet prvků, odpovídající definici funkce. Tedy buď stejný, nebo menší v případě, že funkce má pro některé parametry definovanou výchozí hodnotu. Což má druhá vytvořená funkce s názvem **implicit**.

In [291]:
tu3 = (10, 30)
implicit(*tu3)

{'c': 'retezec', 'b': 30, 'a': 10}


A k čemu se tento magický zápis hodí? V definici funkcí zejména tam, kde potřebujeme vytvořit dostatečně široké rozhraní, které zajistí předání všech parametrů. Tedy spíše u systémových věcí, jako je volání konstruktoru rodičovské třídy, nebo při psaní dekorátorů. O tom ale ještě bude řeč. 

Druhý způsob použití, tedy rozbalení sekvence je možná častější. Odpovídá vlastně vícenásobnému přiřazení. I když v tomto případě dělá "magii" Python automaticky, není potřeba používat hvězdičky. Pro zopakování se podívejte na následující příklad.

In [292]:
a, b, c = tu2
c

30

## Inplace operace a měnitelné typy

Pozor na to, že některé metody a operace definované pro měnitelné typy jako je **list** tento objekt opravdu modfikují. Právě proto, že je to modifikovatelný neboli měnitelný nebo pěkně česky mutable typ. 

Vytvoříme jednoduchý list a zjistíme si jeho id.

In [293]:
li = [10, 20, 30]
id(li)

139862661438664

In place operace - tedy připojení dalšího listu pomocí operátoru += modifikuje původní list.

In [294]:
li += [30, 40]
li

[10, 20, 30, 30, 40]

Což si můžeme potvrdit tím, že se opět podíváme na jeho id.

In [295]:
id(li)

139862661438664

Pokud ale použijeme běžné přičítání, jde o operaci, která původní list nemění. Následující zápis totiž doslova interpretu jazyka říká: "vezmi data uložená pod jménem **li**, připoj k nim následující data a výsledek  ulož znovu pod jménem **li**. Stará data pak můžeš zahodit."

In [296]:
li = li + [50, 60]
li

[10, 20, 30, 30, 40, 50, 60]

Výsledkem je tedy opět list, ale už má jiné id.

In [297]:
id(li)

139862662046984

Podobně modifkuje **list** také jeho metoda **append**. To asi není moc překvapivé, ale je dobré si pamatovat, že metoda sice modifkuje list, ale sama o sobě nevrací žádnou hodnotu - vrací NoneType.

In [298]:
li.append(100)
li

[10, 20, 30, 30, 40, 50, 60, 100]

Jen pro kontrolu - id je stejné.

In [299]:
id(li)

139862662046984

Začátečník, no vlastně nejen ten, občas chce výsledek operace append použít jako nový list. A tady nastává problém

In [300]:
x = li.append(333)
li

[10, 20, 30, 30, 40, 50, 60, 100, 333]

List se sice modifikoval, ale výsledek x je prázdný. Metoda append totiž modifikuje objekt, nevrací jeho kopii.

In [301]:
x

In [302]:
type(x)

NoneType

Obdobně se chovají i další metody objektu **list** například **sort**.

In [303]:
y = li.sort(reverse=True)
type(y)

NoneType

In [304]:
li

[333, 100, 60, 50, 40, 30, 30, 20, 10]

Takže pokud opravdu potřebujeme výsledek jako novou proměnnou a původní list naopak nechceme měnit, musíme si poradit jinak. V případě řazení poslouží funkce sorted() jak už dobře víte.

In [305]:
li

[333, 100, 60, 50, 40, 30, 30, 20, 10]

In [306]:
y = sorted(li)
y

[10, 20, 30, 30, 40, 50, 60, 100, 333]

## Generátory 

Generátor je objekt, který umí generovat další objekty podobných vlastností. Například prvky matematické posloupnosti, nebo list na jehož prvky aplikujeme nějakou funkci. Generátory mohou být jak **konečné**, tak i **nekonečné**. Ty druhé generují data dokud běží program. Výhodou generátorů je, že jsou paměťově velmi efektivní a tudíž i rychlé.

Pro vytvoření konečného generátoru můžeme použít generátorovou notaci podobně jako u generátoru seznamu či slovníku. Pouze v tomto případě využije kulatých závorek, ne lomených (list) či složených (dict).

Následující zápis už znáte - vygeneruje list druhých mocnin čísel 0-9-

In [307]:
ligen = [t**2 for t in range(10)]
ligen

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [308]:
type(ligen)

list

Generátor z něj vytvoříme změnou dvou znaků - závorek:

In [309]:
gen = (t**2 for t in range(10))
gen

<generator object <genexpr> at 0x7f345036db48>

In [310]:
type(gen)

generator

Pokud nás zajímají všechna čísla z posloupnosti najednou, je list správná struktura. Co když ale hledáme pouze maximum? Nebo průměr? Nebo součet celé pousloupnosti. V takovém případě je zbytečné nejprve generovat list a držet ho celý v paměti.

In [311]:
sum(gen)

285

In [312]:
sum(ligen)

285

Pozor na to, že konečný generátor se "vyčerpá" pokud projdeme všechny jeho prvky. Pokud je potřeba opakovaně, musíme ho také opakovaně vytvořit. Případně implementovat vlastní typ, který to udělá automaticky, ale o tom příště. 

Iterovat přes generátor nemusíme jen pomocí cyklu for. Další prvek vydá generátor když na něj zavoláme funkci next(). Můžeme tedy iterovat i jinak, třeba pomocí cyklu while.

In [313]:
gen = (t**2 for t in range(10))
run = True
while run:
    prvek = next(gen)
    if prvek > 33:
        run = False
    
    print(prvek)

0
1
4
9
16
25
36


Pokud nám z nějakého důvodu generátorová notace nestačí či nevyhovuje, můžeme použít klíčové slovo **yield** a vytvořit generátor pomocí definice funkce. **Yield** použijeme tam, kde v běžné funkci bude return. Tedy skoro. 

Return totiž funkci ukončí, takže kód který následuje za ním se nikdy neprovede (ponechme stranou různá větvení). V případě **yield**, ale přijde tento kód při generování na řadu. 

Následující jednoduché počítadlo je implementováno jako nekonečný generátor. 

In [314]:
def make_counter(x):
    print('entering make_counter')
    while True:
        yield x
        print('incrementing x')
        x = x + 1

Nekonečné generátory je samozřejmě lepší volat pomocí funkce **next**. U for cyklu se nemusíte dočkat konce :)

Vytvoříme si tedy počítadlo s úvodní hodnotou 1. Po jeho aktivaci pomocí funkce next() dojde nejprve k voláni funkce print('entering...') a pak už **yield** vydá první hodnotu. Kód funkce se následně zastaví.

In [315]:
counter = make_counter(1)
next(counter)

entering make_counter


1

Jakmile zavoláme next(gen) znovu, kód pokračuje od místa, kde se předtím zastavil. Na dalším řádku je opět funkce print a potom dojde k inkrementaci hodnoty x. While cyklus pokračuje dál, takže opět dojdeme k volání **yield**. Funkce vydá hodnotu a opět se zastaví.

In [316]:
next(counter)

incrementing x


2

A tak dále. Dokud neukončíme celý program, nebo objekt generátoru neukončíme jinak. 

In [317]:
next(counter)

incrementing x


3

To platí pro nekonečný generátor. Konečný generátor dojde po předem určeném počtu iterací na konec, což dá najevo vyvoláním výjimky **StopIteration**. For cyklus podle ní pozná, kdy iteraci ukončit, v případě jiného způsobu iterace musíte tuto výjimku zachytit a ošetřit.